# Business Analytics und Künstliche Intelligenz

Prof. Dr. Jürgen Bock & Maximilian-Peter Radtke

---

In [ ]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

## Daten zur Kreditausfallwahrscheinlichkeit

In [ ]:
default = pd.read_csv('Default.csv')

In [ ]:
default.head()

In [ ]:
default['intercept'] = 1
default.default = default.default.map({'Yes': 1, 'No': 0})
default.student = default.student.map({'Yes': 1, 'No': 0})

In [ ]:
default.head()

In [ ]:
for col in default.columns:
    plt.title(col)
    plt.hist(default[col])
    plt.show()

## Logistische Regression

In [ ]:
defaultMod = sm.Logit(default.default, default.loc[:,['balance', 'intercept']])
defaultRes = defaultMod.fit()
defaultRes.summary()

In [ ]:
defaultStudMod = sm.Logit(default.default, default.loc[:,['student', 'intercept']])
defaultStudRes = defaultStudMod.fit()
defaultStudRes.summary()

In [ ]:
defaultAllMod = sm.Logit(default.default, default.loc[:,['student', 'balance', 'income', 'intercept']])
defaultAllRes = defaultAllMod.fit()
defaultAllRes.summary()

## Lineare Diskriminanzanalyse (LDA)

Statsmodels bietet [LDA]( 'https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html') leider nicht als Modell an. Daher nutzen wir hierfür das Paket [`scikit-learn`]('https://scikit-learn.org/stable/index.html'). Dieses Paket enthält eine vielzahl von Modellen und Methodiken, welche für maschinelles Lernen genutzt werden können. Das Paket ist mehr auf die Anwendung von maschinellem Lernen spezialisert und es werden daher nicht automatisch so viele statistsiche Informationen ausgegeben wie bei `statsmodels`. Die Dokumentation bietet sehr viel anschauliche Beispiele, wie einzelne Modelle genutzt werden können.

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
ldaDef = LinearDiscriminantAnalysis()
ldaDef.fit(default.drop('default', axis=1),default.default)

In [ ]:
ldaDef.score(default.drop('default', axis=1),default.default)

Das Paket bietet auch weitere Metriken, um die Güte von Modellen zu betrachten, beispielsweise die [Confusionmatrix]("https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html") aus der Vorlesung.

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
yHatLDA = ldaDef.predict(default.drop('default', axis=1))
confMat = confusion_matrix(default.default, yHatLDA)
confMat

## K-Nearest-Neighbors (KNN)

Auch [KNN]('https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html') ist als Modell bei `scikit-learn` hinterlegt.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knnClf = KNeighborsClassifier(n_neighbors=3)
knnClf.fit(default.drop('default', axis=1), default.default)

In [ ]:
yHatKNN = knnClf.predict(default.drop('default', axis=1))
confMat = confusion_matrix(default.default, yHatKNN)
confMat

Was könnte der Grund dafür sein, dass LDA in diesem Fall schlechter als KNN abschneidet?

In [ ]:
for i in [0,1]:
    for col in ['balance', 'income']:
        plt.hist(default.loc[default.default == i, col])
        plt.title(f'Class {i} Feature {col}')
        plt.show()

---

# Übungsaufgaben

Sklearn bietet auch die Möglichkeit kleine Datensätze direkt über das Paket herunterzuladen, beispielsweise den Datensatz zur Klassifizierung von Schwertlilien, welchen wir schon in der ersten Vorlesung gesehen haben.

In [ ]:
from sklearn.datasets import load_iris
data = load_iris()

Nutzen Sie den Irisdatensatz, um die folgenden Aufgaben zu lösen.

## Aufgabe 1

Teilen Sie die Daten in einen Test- und einen Trainingsdatensatz mit der [`train_test_split`]("https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html") Funktion aus sklearn auf. Nutzen Sie ein Verhältnis von 80% Traingsdaten und 20% Testdaten.

In [ ]:
data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = data['data'] 
y = data['target']

In [ ]:
XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=0.2, random_state=4)

In [ ]:
XTrain.shape

In [ ]:
XTest.shape

## Aufgabe 2

Trainieren Sie jeweils einen LDA- und KNN-Klassifikator auf Basis der Trainingsdaten. Bewerten Sie die Ergebnisse anhand der Confusion Matrix auf Basis der Test- bzw. Traingsdaten. Welcher Klassifikator schneidet am besten ab?

In [ ]:
ldaClf = LinearDiscriminantAnalysis()
ldaClf.fit(XTrain, yTrain)
ldaClf.score(XTest, yTest)

In [ ]:
ldaClf.score(XTrain, yTrain)

In [ ]:
confMatLda = confusion_matrix(yTest, ldaClf.predict(XTest))
confMatLda

In [ ]:
knnClf = KNeighborsClassifier(n_neighbors=7)
knnClf.fit(XTrain, yTrain)
knnClf.score(XTest, yTest)

In [ ]:
confMatKnn = confusion_matrix(yTest, knnClf.predict(XTest))
confMatKnn

## Aufgabe 3

Betrachten Sie nun nochmals die  Modelle, um die Kreditausfallwahrscheinlichkeit vorherzusagen (logistische Regression / LDA / KNN). Trainieren Sie diese erneut, aber mit der Hilfe eines Train Test Splits. Berechenen Sie zu diesen neuen Modellen jeweils die Metriken Recall, Precision und F1 Score. Nutzen Sie dazu die Funktionen von Sklearn. Welches Modell ist Ihrer Meinung nach das Beste?

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score

In [ ]:
XTrain_default, XTest_default, yTrain_default, yTest_default = train_test_split(default.drop('default', axis=1), default.default, test_size=0.2, random_state=4)

In [ ]:
defaultAllMod = sm.Logit(yTrain_default, XTrain_default.loc[:,['student', 'balance', 'income', 'intercept']])
defaultAllRes = defaultAllMod.fit()
print(f'Recall: {recall_score(yTest_default, defaultAllRes.predict(XTest_default).round())}')
print(f'Precision: {precision_score(yTest_default, defaultAllRes.predict(XTest_default).round())}')
print(f'F1: {f1_score(yTest_default, defaultAllRes.predict(XTest_default).round())}')

In [ ]:
ldaClf = LinearDiscriminantAnalysis()
ldaClf.fit(XTrain_default, yTrain_default)
print(f'Recall: {recall_score(yTest_default, ldaClf.predict(XTest_default))}')
print(f'Precision: {precision_score(yTest_default, ldaClf.predict(XTest_default))}')
print(f'F1: {f1_score(yTest_default, ldaClf.predict(XTest_default))}')

In [ ]:
knnClf = KNeighborsClassifier(n_neighbors=3)
knnClf.fit(XTrain_default, yTrain_default)
print(f'Recall: {recall_score(yTest_default, knnClf.predict(XTest_default))}')
print(f'Precision: {precision_score(yTest_default, knnClf.predict(XTest_default))}')
print(f'F1: {f1_score(yTest_default, knnClf.predict(XTest_default))}')